### Coleta de dados

Este notebook tem por finalidade realizar a coleta de dados que será utilizada no estudo de hiperparâmetros

#### Dados utilizados

* Ações do Facebook (FB)
* Ações da Petrobras (PBR)
* Ações da Apple (AAPL)
* Ações da Tesla (TSLA)

Serão armazenadas apenas as variáveis de abertura e fechamento para os períodos de 1, 3 e 5 anos

In [2]:
import pandas as pd
import pandas_datareader.data as web
import datetime as dt

In [9]:
ativos = ['FB', 'PBR', 'AAPL', 'TSLA']

datas_inicio = [dt.datetime(2016, 3, 1), dt.datetime(2018, 3, 1), dt.datetime(2020, 3, 1)]

data_fim = dt.datetime(2021, 3, 1)

In [7]:
end = dt.datetime(2020, 6, 1)
start = dt.datetime(2019, 1, 1)

df = web.DataReader("FB", 'yahoo', start, end)


df = df.reset_index()
df = df.drop(columns=['Date','High','Low','Volume','Adj Close'])

nomes = {'Close': 'Alvo',
         'Open': 'Abertura'}

df = df.rename(columns = nomes)

356


,Abertura,Alvo
0,128.990005,135.679993
1,134.690002,131.740005
2,134.009995,137.949997
3,137.559998,138.050003
4,139.889999,142.529999


In [17]:
for at in ativos:
    
    for i, dt_i in enumerate(datas_inicio):
        
        if i == 0:
            periodo = 5
        elif i == 1:
            periodo = 3
        elif i == 2:
            periodo = 1
            
        df = web.DataReader(at, 'yahoo', dt_i, data_fim)
        
        df = df.reset_index()
        df = df.drop(columns=['Date','High','Low','Volume','Adj Close'])

        nomes = {'Close': 'Alvo',
                 'Open': 'Abertura'}

        df = df.rename(columns = nomes)
        
        file = "Datasets/"+str(at)+"_"+str(periodo)+'.csv'
        
        df.to_csv(file)
        print(file)

Datasets/FB_5.csv
Datasets/FB_3.csv
Datasets/FB_1.csv
Datasets/PBR_5.csv
Datasets/PBR_3.csv
Datasets/PBR_1.csv
Datasets/AAPL_5.csv
Datasets/AAPL_3.csv
Datasets/AAPL_1.csv
Datasets/TSLA_5.csv
Datasets/TSLA_3.csv
Datasets/TSLA_1.csv
